# AOC 2022

In [537]:
\cd notebooks

'io



# Advent of Code 2022 - 04

URL: https://adventofcode.com/2022/day/4

We'll study ngn's [solution](https://codeberg.org/ngn/k/src/branch/master/aoc/22/04.k):

In [538]:
`0:$(+/1>*/-/)'1{|y}\\`I$+'"-"\''+","\'0:"i/04" /both parts

538
792


This one is "simple". Catch full overlaps of 2 ranges.
As always, first, the input:

In [551]:
10#x:0:"i/04" 

("5-96,6-99"
 "29-97,80-97"
 "3-87,3-4"
 "12-12,6-12"
 "64-65,24-66"
 "52-54,1-53"
 "86-89,85-88"
 "37-69,38-38"
 "81-96,47-75"
 "1-13,18-44")


Split each line by comma.  The nice mnemonic I use to distinguish `\` split and `/` join, is mapping it to scan `\` vs over `/`. The backslash, leans backward, and leaves trails of the computation.

So, backslash, like scan, and like split produces many results.

On the other hand, forward slash is "moving forward", and it doesn't need to keep intermediate results, so it returns a single result (like join `/`, which also returns one result).

In [540]:
10#","\'x

(("5-96";"6-99")
 ("29-97";"80-97")
 ("3-87";"3-4")
 ("12-12";"6-12")
 ("64-65";"24-66")
 ("52-54";"1-53")
 ("86-89";"85-88")
 ("37-69";"38-38")
 ("81-96";"47-75")
 ("1-13";"18-44"))


Flip (`+`) the table, so we get all first pairs in the first row and the second in the second.
Foreach string, do another split by "-".

And foreach of them, flip again

In [561]:
xx:`I$+'"-"\''+","\'x

This partial result is a nested list with the shape:

In [562]:
:xxx:(10#)''xx

((5 29 3 12 64 52 86 37 81 1;96 97 87 12 65 54 89 69 96 13)
 (6 80 3 6 24 1 85 38 47 18;99 97 4 12 66 53 88 38 75 44))


I was a bit puzzled by the following double scan but first Let's figure out the simplest possible solution to the problem, and we'll figure out the double scan later.

For a range to be fully contained in another range, there are only 3 possibilities:
- first range is strictly bigger on both sides
- second range is strictly bigger on both sides
- they share one limit.


Here, ngn relies on the fact that if range `a` (a1-a2) is overflows `b` (b1-b2) in both ends:
`a1-b1>0  and a2-b2<0`. 

If b would be larger, `a1-b1<0  and a2-b2>0`. The property that holds in both is that one is positive and the other negative. So multiplying them, should give us a negative number.

The case of sharing a limit is funny, because if you share one of the ends, it means that you fully overlap, no matter if `a` is bigger or `b`. It counts as an overlap.

That's why `(1>*/-/)` over the ranges will signal all fully overlapping ranges. If `1>` means that either the multiplication is negative (meaning full strict overlap) or 0 (at least 1 side matches). 

Here are some intermediate results to help with the calculations:

In [555]:
xxx
(-/)xxx
(*/-/)xxx
(1>*/-/)xxx

((5 29 3 12 64 52 86 37 81 1;96 97 87 12 65 54 89 69 96 13)
 (6 80 3 6 24 1 85 38 47 18;99 97 4 12 66 53 88 38 75 44))
(-1 -51 0 6 40 51 1 -1 34 -17
 -3 0 83 0 -1 1 1 31 21 -31)
3 0 0 0 -40 51 1 -31 714 527
0 1 1 1 1 0 0 1 0 0


In [544]:
(+/1>*/-/)xx

538


Second part is to count the pairs of ranges that overlap at all. Here, the property that the solution uses is the following:

- ( a1-b2 * a2-b1 ) < 1   -> overlap

This means that the formula is very similar, but instead of comparing a's with a's and b's with b's, we do the subtraction in a crossed way.

```
a1....a2
   b1.....b2
```

K-wise, the trick is to swap b1 and b2 columns, and repeat the same operation. 


And here, the cherry on top: `n d\\l`.

To make it more "fun", here ngn uses this reverse-scan trick, that also got me thinking for a while.

It's a nice opportunity to talk in depth about this `1{|y}\\` pattern:

First let's look at `{|y}\` in isolation:

It follows `d\l`

In scan dyad, the return value will have the same length as the list `l`. In case the list has 2 elements, the result will be:
`(l@0 ; l@0 d l@1 )`.

In this particular case, ngn wants to apply a function to the second element (`l@1`)

In [556]:
{|y}\2 5#!20

(0 1 2 3 4
 9 8 7 6 5)


If instead of being a dyad, `f` is a monad, the function is applied to the full `l`. not in an elemet by element basis. So

`n m\l` is:

`(l; f l; f f l; ....)` n times.

Now is the moment where both patterns are combined:

`n d\\l` can be seen as `n m\l` with m=`d\`.

`n d\\l` return value has the same lenght as `l`, the first which is the original `l`, and the second gets m applied to it.  

In this case, `m` is `\d`, which returns the same number of elements of its parameter (so, of the original `l`), with the first element being the original one (`l@0`, and the second one being `l@0 d l@1`. In this case, d is `{|y}`.

Datawise, 1{|y}\\(a;b) -> `((a;b);(a;|b))`




In [571]:
{|y}\\2 2 5#!20  /simplified with `int`s

(((0 1 2 3 4;5 6 7 8 9);(10 11 12 13 14;15 16 17 18 19))
 ((0 1 2 3 4;5 6 7 8 9);(15 16 17 18 19;10 11 12 13 14)))


In [572]:
1{|y}\\xxx   /sample with our real data

(((5 29 3 12 64 52 86 37 81 1;96 97 87 12 65 54 89 69 96 13);(6 80 3 6 24 1 85 38 47 18;99 97 4 12 66 53 88 38 75 44))
 ((5 29 3 12 64 52 86 37 81 1;96 97 87 12 65 54 89 69 96 13);(99 97 4 12 66 53 88 38 75 44;6 80 3 6 24 1 85 38 47 18)))


In [581]:
`0:$(+/1>*/-/)'1{|y}\\`I$+'"-"\''+","\'0:"i/04" /both parts

538
792
